# Pour obtenir les données de synthea : 

Étapes exactes

0. Inutile de cloner le dépôt synthea !

    ( Cela a déjà été fait dans un autre répertoire que celui de AURA, puis les références git ont été supprimées )

    ( à l'origine ça venait de : git clone https://github.com/synthetichealth/synthea.git )


1. se placer dans le bon répertoire :

    cd synthea


2. Utiliser le Dockerfile dans ce dossier :


In [ ]:
FROM openjdk:11

# Installer git et unzip
RUN apt-get update && apt-get install -y git unzip && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

# Copier le code synthea dans le conteneur
COPY . /synthea
WORKDIR /synthea

# Rendre le script gradlew exécutable (au cas où)
RUN chmod +x ./gradlew

# Configurer les variables d'environnement pour Java/Gradle
ENV GRADLE_OPTS="-Xmx2g -Xms1g -XX:MaxMetaspaceSize=512m"
ENV JAVA_OPTS="-Xmx4g -Xms2g -XX:+UseG1GC"

# Build du projet Java en sautant les tests pour éviter les problèmes de mémoire
# Synthea est un générateur de données Java, les tests peuvent être lourds
RUN ./gradlew build -x test --no-daemon --max-workers=1

# Alternative avec tests (si vous avez assez de mémoire) :
# RUN ./gradlew build --no-daemon --max-workers=1 -Dorg.gradle.jvmargs="-Xmx4g -Xms2g"

# Vérifier que le JAR a été créé
RUN ls -la build/libs/

# Générer les patients synthétiques au lancement
# 1000 patients ça doit générer 1000000 observations, regroupées par la suite en 50000 lignes
# 50 patients génère ? observations, regroupées en ? lignes
CMD ["java", "-jar", "build/libs/synthea-with-dependencies.jar", "-p", "50"]

Dans le dossier SYNTHEA/src/main/resources, modifier le fichier synthea.properties pour :
 
   
    exporter.fhir.export = true
    exporter.csv.export = true
    exporter.text.export = true
    exporter.notes.fhir.export = true # ?? peut être que ça ne génère rien

3. Construire l’image (va demander à télécharger JAVA) :

    docker build -t synthea-100k-from-src .


4. Lancer la génération vers ./GENERATED_FILES :

    mkdir -p ../GENERATED_FILES

    docker run --rm -v "$(pwd)/../GENERATED_FILES":/synthea/output synthea-100k-from-src


In [ ]:
# première commande qui fonctionnait :
# docker run --rm -v "$(pwd)/../GENERATED_FILES":/synthea/output synthea-100k-from-src


# proposition de claude.ai : 
# docker run --rm -v "$(pwd)/../GENERATED_FILES":/synthea/output -v "$(pwd)/src/main/resources/synthea.properties":/synthea/src/main/resources/synthea.properties synthea-100k-from-src

In [2]:
# Tester la présence des fichiers
from pathlib import Path

current_dir = Path.cwd()
parent = current_dir.parent

data_dir : Path = None
for path_info in parent.iterdir():
    if path_info.is_dir() and path_info.name=='DATA' :
        data_dir = path_info
        break

if data_dir != None :
    print(f'Répertoire : {data_dir}')


fhir_dir = Path(f'{data_dir}/GENERATED_FILES/fhir')
fichiers_json = list(fhir_dir.glob('*.json'))
print(f"Fichiers JSON : {len(fichiers_json)} = pas loin de 50")

csv_dir = Path(f'{data_dir}/GENERATED_FILES/csv')
fichiers_csv = list(csv_dir.glob('*.csv'))
print(f"Fichiers CSV : {len(fichiers_csv)} = pas loin de 18")


Répertoire : /home/nicolascassonnet/Documents/WORK/aura-clinical-nlp/DATA
Fichiers JSON : 56 = pas loin de 50
Fichiers CSV : 18 = pas loin de 18


5. Générer un texte explicatif (en anglais) pour chaque observation

    ... en cours : 
  
    voir le fichier generate_annoted_csv.ipynb


In [ ]:
# from transformers import pipeline

# translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")

# def translate_fr_to_en(text_fr):
#     result = translator(text_fr, max_length=512)
#     return result[0]['translation_text']

In [ ]:
# import pandas as pd
# from transformers import pipeline
# from sentence_transformers import SentenceTransformer, util

# # 1. Traduction
# translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
# def translate_fr_to_en(text_fr):
#     result = translator(text_fr, max_length=512)
#     return result[0]['translation_text']

# # 2. Charger Synthea
# patients = pd.read_csv("synthea_output/csv/patients.csv")
# conditions = pd.read_csv("synthea_output/csv/conditions.csv")
# encounters = pd.read_csv("synthea_output/csv/encounters.csv")

# # 3. Préparer un texte descriptif par patient (concaténation conditions + encounters)
# def get_patient_text(pid):
#     conds = conditions[conditions['patient_id'] == pid]['description'].fillna('').str.cat(sep=' ')
#     encs = encounters[encounters['patient_id'] == pid]['description'].fillna('').str.cat(sep=' ')
#     return conds + ' ' + encs

# patients['text_data'] = patients['id'].apply(get_patient_text)

# # 4. Embeddings
# model = SentenceTransformer('all-MiniLM-L6-v2')

# patient_embeddings = model.encode(patients['text_data'].tolist(), convert_to_tensor=True)

# def find_top_patients(text_en, top_k=3):
#     query_emb = model.encode(text_en, convert_to_tensor=True)
#     cos_scores = util.cos_sim(query_emb, patient_embeddings)[0]
#     top_results = cos_scores.topk(k=top_k)
#     return [(patients.iloc[idx]['id'], cos_scores[idx].item()) for idx in top_results.indices]

# # 5. Utilisation
# text_fr = "Fièvre élevée, toux sèche, difficulté à respirer."
# text_en = translate_fr_to_en(text_fr)

# top_patients = find_top_patients(text_en)

# for pid, score in top_patients:
#     print(f"Patient {pid} - Score similarité: {score:.3f}")
#     conds = conditions[conditions['patient_id'] == pid][['description', 'start_date', 'end_date']]
#     print("Conditions associées :")
#     print(conds.head())